In [1]:
import re
import csv
import json
import pandas as pd
import os
from camel_tools.utils import charsets
import xml.etree.ElementTree as et
import sys
import subprocess
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('/Users/nurpeiis/Desktop/Capstone/hierarchical-did/utils')
from data_process import DataProcess

In [7]:
# Step 0: Preprocess testset
df_test = pd.read_csv('../../data_raw/qadi_country/testset/QADI_test.txt', delimiter='\t', header=None)

In [8]:
df_test.to_csv('../../data_raw/qadi_country/test_processed.tsv', sep='\t', header=None, index=False)    

In [9]:
# Step 1: Preprocess json trainset
files = [i for i in os.listdir('../../data_raw/qadi_country/train_json') if i[-4:] == 'json']
df = pd.DataFrame(columns={'original_sentence', 'dialect_country_id'})
for file in files:
    dialect = file[15:-5]
    with open('../../data_raw/qadi_country/train_json/' + file) as f:
        lines = f.read().splitlines()
    final = []
    sentences = []
    for l in lines:
        sentences.append(' '.join(json.loads(l)['full_text'].split()))
    df_temp = pd.DataFrame(columns={'original_sentence', 'dialect_country_id'})
    df_temp['original_sentence'] = sentences
    df_temp['dialect_country_id'] = dialect
    df = df.append(df_temp, ignore_index=True)


In [10]:
df

,original_sentence,dialect_country_id
0,@naif_su جيد أنه أعجبك وهنيئا لك والناس يختلفو...,LB
1,@Mef_4500 @_Health_8 ما فهمت الموضوع عفوا,LB
2,@iS5Pym1qiiVtGbF صحيح، هو بدون سكر لكن فيه مُح...,LB
3,@haso1181 صحيح يا غالي وهو متوفر من فترة في مح...,LB
4,@wail_alhazmi ربي يسلمك ويكرمك ويرفع قدرك ويسع...,LB
...,...,...
496460,@lwmUMSZ8BOTH1tD @Rami_Dalati قلنالو بس ما يست...,SY
496461,رامي دالاتي ليس اشرف من عبدالجليل السعيد واحد ...,SY
496462,رامي دالاتي كلب جعاري https://t.co/zCHCgOP2Jp,SY
496463,@hh66063383 @Rami_Dalati ابو اسلام اذا امك شرم...,SY


In [11]:
df.to_csv('../../data_raw/qadi_country/train_processed.tsv', index=False, sep='\t')

In [15]:
dp = DataProcess('../data_processed_splited/qadi/', 'user_level', 'twitter', 'http://alt.qcri.org/resources/qadi/', 'qadi', {},{},1, 'corpus', 'original')
dp.save_file('train_processed.tsv', dp.preprocess('../../data_raw/qadi_country/train_processed.tsv', '', '', 1, '', header=0))

In [3]:
dp = DataProcess('../data_processed_splited/qadi/', 'user_level', 'twitter', 'http://alt.qcri.org/resources/qadi/', 'qadi', {},{},0, 'corpus', 'original')
dp.save_file('test_processed.tsv', dp.preprocess('../../data_raw/qadi_country/test_processed.tsv', '', '', 1, '', header=0))

In [16]:
dp.standardize_labels('../data_processed_splited/qadi/test_processed.tsv', '../data_processed_splited/qadi/test_processed.tsv', ['country'])
dp.standardize_labels('../data_processed_splited/qadi/train_processed.tsv', '../data_processed_splited/qadi/train_processed.tsv', ['country'])



../data_processed_splited/qadi/test_processed.tsv
Unique cities  [nan]
Unique provinces  [nan]
Unique countries  ['ye' 'iq' 'bh' 'sy' 'ps' 'lb' 'ae' 'sa' 'msa' 'om' 'tn' 'jo' 'kw' 'sd'
 'qa' 'ma' 'eg' 'dz' 'ly']
Unique regions  ['gulf_aden' 'iraq' 'gulf' 'levant' 'msa' 'maghreb' 'nile_basin']
../data_processed_splited/qadi/train_processed.tsv


/Users/nurpeiis/miniconda3/envs/hierarchical-did/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Unique cities  [nan]
Unique provinces  [nan]
Unique countries  ['lb' 'jo' 'eg' 'iq' 'tn' 'kw' 'sd' 'ye' 'ma' 'bh' 'ps' 'om' 'qa' 'ly'
 'ae' 'sa' 'dz' 'sy']
Unique regions  ['levant' 'nile_basin' 'iraq' 'maghreb' 'gulf' 'gulf_aden']


In [17]:
dp.save_features("../datasets_splited_features.tsv")

In [13]:
#Change Lybian region to Maghreb rather than Levant
folder = '../data_processed/qadi/'
files = os.listdir(folder)
for file in files:
    df = pd.read_csv(f'{folder}{file}', delimiter='\t', header=0)
    for index, row in df.iterrows():
        if row['dialect_country_id'] == 'ly':
            df.at[index,'dialect_region_id'] = 'maghreb'
    df.to_csv(f'{folder}{file}', sep='\t', index=False)

In [ ]:
df[['dialect_country_id', 'dialect_region_id']].drop_duplicates()